In [ ]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

In [ ]:
f = open("a_b_zeta.txt", "r")
lines = f.readlines()

In [ ]:
# 비밀키
a = [ ]

# 암호문
b = []

# zeta
zeta = []

for line in lines:
    temp = []
    for i, value in enumerate(line.split(' ')):
        if i<4:
            # a 값을 저장
            temp.append(value)
            if i == 3:
                a.append(temp)
                temp = []
        elif i<8:
            # b 값을 저장
            temp.append(value)
            if i == 7:
                b.append(temp)
                temp = []
        else:
            # zeta 값을 저장
            temp.append(value)
            if i == 9:
                zeta.append(temp)
                temp = []

In [ ]:
a = np.array(a)
b = np.array(b)
zeta = np.array(zeta)

In [ ]:
print(a.shape)
print(b.shape)

In [ ]:
temp = []

for i in range(a.shape[0]):
    temp_a = []
    for index, value in enumerate(a[i]):
        if index < 2:
            if index == 0:
                a_1 = np.int16(0)
                a_1 += (np.int16(int(value, 16))<<8)
            else:
                a_1 += (np.int16(int(value, 16)))
                temp_a.append(a_1)
        else:
            if index == 2:
                a_2 = np.int16(0)
                a_2 += (np.int16(int(value, 16))<<8)
            else:
                a_2 += (np.int16(int(value, 16)))
                temp_a.append(a_2)
    temp.append(temp_a)

a = np.array(temp, dtype = 'int16')
a.shape

In [ ]:
temp = []

for i in range(b.shape[0]):
    temp_b = []
    for index, value in enumerate(b[i]):
        if index < 2:
            if index == 0:
                b_1 = np.int16(0)
                b_1 += (np.int16(int(value, 16))<<8)
            else:
                b_1 += (np.int16(int(value, 16)))
                temp_b.append(b_1)
        else:
            if index == 2:
                b_2 = np.int16(0)
                b_2 += (np.int16(int(value, 16))<<8)
            else:
                b_2 += (np.int16(int(value, 16)))
                temp_b.append(b_2)
    temp.append(temp_b)
b = np.array(temp, dtype = 'int16')
b.shape

In [ ]:
temp = []

for i in range(zeta.shape[0]):
    temp_b = []
    for index, value in enumerate(zeta[i]):
        if index < 2:
            if index == 0:
                b_1 = np.int16(0)
                b_1 += (np.int16(int(value, 16))<<8)
            else:
                b_1 += (np.int16(int(value, 16)))
                temp_b.append(b_1)
        else:
            if index == 2:
                b_2 = np.int16(0)
                b_2 += (np.int16(int(value, 16))<<8)
            else:
                b_2 += (np.int16(int(value, 16)))
                temp_b.append(b_2)
    temp.append(temp_b)
zeta = np.array(temp, dtype = 'int16')
zeta.shape

In [ ]:
def montgomery_reduce(a):
    # print(a)
    t = np.int16(0)
    # print("a type", type(a))
    # print("int16_t a ", np.int16(a))
    t = np.int16(a) * np.int16(-3327)
    # print("(int16_t)a * QINV  %d" % t)
    # print("(int32_t)t*KYBER_Q %d" % (np.int32(t) * np.int32(3329)))
    t1 = np.int32(t) * np.int32(3329)
    # print("(a - (int32_t)t*KYBER_Q) %d" %(np.int32(a) - t1))
    t = ((np.int32(a)-(np.int32(t) * np.int32(3329))) >> 16)
    t = np.int16(t)
    # print("t %d" %t)
    return t

def fqmul(a, b):
    # print(type(a))
    # print("a * b = ", np.int32(a) * np.int32(b))
    return montgomery_reduce(np.int32(a) * np.int32(b))

# a = 키
# b = 암호문
def basemul(a, b, zeta):
    r = np.array([0, 0], dtype = 'int16')
    r[0]  = fqmul(np.int16(a[1]), np.int16(b[1]));
    return r[0]
    r[0]  = fqmul(r[0], zeta);
    r[0] += fqmul(a[0], b[0]);
    r[1]  = fqmul(a[0], b[1]);
    r[1] += fqmul(a[1], b[0]);
    return r

In [ ]:
def haming_weight(number):
    cnt = 0
    check = 0x1
    for _ in range(16):
        if (number & check):
            cnt +=1
            check = (check << 1)
        else:
            check = (check << 1)
    return cnt

# hw = np.zeros(shape=(0xFFFF,5000))

# # i가 키다
# for i in tqdm(range(0xFFFF)):
#     # j가 평문의 개수
#     for j in range(5000):
#         hw[i][j] = haming_weight(result[i][j])

In [ ]:
# def basemul(a, b, zeta):
#     r = np.array([0, 0], dtype = 'int16')
#     r[0]  = fqmul(np.int16(a[1]), np.int16(b[1]));
#     return r[0]
#     r[0]  = fqmul(r[0], zeta);
#     r[0] += fqmul(a[0], b[0]);
#     r[1]  = fqmul(a[0], b[1]);
#     r[1] += fqmul(a[1], b[0]);
#     return r

def fast_fqmul(key, b):
    # r[0] 구하는 단계
    key = np.int16(key)
    b = np.int16(b)
    t2 = np.int32(key) * np.int32(b)
    t3 = np.int16(t2) * np.int16(-3327)
    t4 = np.int32(t3) * np.int32(3329)
    t5 = ((np.int32(t2) - (np.int32(t3) * np.int32(3329))) >> 16)
    t5 = np.int16(t5)
    return t5


In [ ]:
from tqdm.notebook import tqdm 

# b = b[:10000]

# t2 = cp.int32()
result = np.int16(np.zeros(shape=(0xFFFF,100000)))
vf = np.vectorize(haming_weight)
b_1 = np.array(b[:100000, 1])

for key in tqdm(range(0xFFFF)):
    t2 = np.int32(key) * np.int32(b_1)
    t3 = np.int16(t2) * np.int16(-3327)
    t4 = np.int32(t3) * np.int32(3329)
    t5 = ((np.int32(t2) - (np.int32(t3) * np.int32(3329))) >> 16)
    t5 = np.int16(t5)
    result[key] = t5

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
vf = cp.vectorize(haming_weight)
# re = cp.array(result[:,:5000])
# hw = vf(re)

In [ ]:
result.shape

In [ ]:
hw = np.zeros(shape=(0xFFFF, 100000), dtype = np.int8)
for i in tqdm(range(20)):
    re = cp.array(result[:,i*5000:i*5000+5000])
    hw[:, i*5000:i*5000+5000] = vf(re).get()

In [ ]:
traces = np.load("traces.npy")

In [ ]:
hw_t = hw.T

In [ ]:
print(traces.shape)
print(hw.shape)
print(hw_t.shape)

In [ ]:
cor_result = []
limit = 100000
key_limit = 0xFFFF
for key in tqdm(range(0,key_limit,1000)):
    if key != 65000:
        temp_hw = hw_t[:,key*1:key*1+1000]
        total = cp.concatenate((cp.array(temp_hw[:limit]), cp.array(traces[:limit])), axis = 1)
        cor = cp.corrcoef(total.T)
        # print(cor.shape)
        temp = []
        for i in range(1000):
            temp.append(cor[i][1000:])
        temp = cp.array(temp)
        cor_result.append(temp)
    else:
        pass
        # temp_hw = hw_t[:,65000:].reshape(-1,1)
        # total = cp.concatenate((cp.array(temp_hw[:limit]), cp.array(traces[:limit])), axis = 1)
        # cor = cp.corrcoef(total.T)
        # print(cor.shape)
        # temp = cor[0][1:]
        # temp = cp.array(temp)
        # cor_result.append(temp)

In [ ]:
cor_result = cp.array(cor_result)
cor_result.shape

In [ ]:
cor_result = cor_result.reshape(-1,600)
print(cor_result.shape)

In [ ]:
a

In [ ]:
cor_result.shape

In [ ]:
# plt.figure(figsize=(30,20))

# for i in range(65000):
#     if i==1173:
#         pass
#     else:
#         plt.plot(cor_result[i,:].get(),alpha=0.3, color = "gray")           
# plt.plot(cor_result[1173].get(), label = "1173", color = "red")        
# plt.legend()
# plt.show()

위에서 실제 키에 해당하는 것이 1173이다.<br>
실제 키에서는 대략적으로 110 ~ 140 까지에서 높은 상관 계수를 갖는 것을 볼 수 있다.<br>
따라서, 이러한 범위에서 키와 유사하게 높은 상관 계수를 갖는 것을 찾아 보자.


### 110:140 범위 중에서 상관 계수가 -0.8이상을 갖는 키값을 모두 뽑는다.

In [ ]:
print(cp.where(cp.nanmin(cor_result[:, 110:140], axis = 1) < -0.8))
cor_min_reuslt = cp.where(cp.nanmin(cor_result[:, 110:140], axis = 1) < -0.8)[0]

#### 110:140 범위 중에서 상관 계수가 -0.8 이상을 갖는 값들은 총 9개가 존재한다. 그 중에서 -0.9 이상을 갖는 값이 2개가 존재한다. 이러한 값에서도 실제 키 값이 더욱 높은 상관 계수를 갖는 것을 볼 수 있다.

In [ ]:
cp.nanmin(cor_result[:, 110:140], axis = 1)[[ 1173,  4502,  7831, 11160, 14489, 17818, 21147, 24476, 27805]]

#### 위의 9개의 높은 상관 계수를 갖는 키값들을 그려본다. 모든 키와 비슷한 형태의 모양을 뛰는 것을 볼 수 있다.

In [ ]:
cp.nanargmax(cp.nanmin(cor_result[:, 110:140], axis = 1))

In [ ]:
plt.figure(figsize=(30,18))

for i in range(65000):
    if i==1173:
        pass
    elif i in cor_min_reuslt:
        plt.plot(cor_result[i,:].get(),alpha=0.3)           
plt.plot(cor_result[1173].get(), label = "1173", color = "red")        
plt.legend()
plt.show()

#### 우리는 그 중에서도 상관 계수가 가장 높은 두 개의 가능한 키값을 통해서 뒤의 연산을 진행 해보도록 하겠다. 실제 키 값은 1173이고 두 번째로 상관 계수가 높은 키 값은 4502이다. 이 두개의 성향을 비교 분석해보자.

In [ ]:
plt.figure(figsize=(30,18))
plt.plot(cor_result[1173].get())

In [ ]:
plt.figure(figsize=(30,18))
plt.plot(cor_result[4502].get())

In [ ]:
plt.figure(figsize=(30,18))
plt.plot(cor_result[21370].get())

#### 실제 키값과 유사한 상관 계수를 갖는 값과 상관 계수가 가장 작은 값에 있어서 HW 분포도는 거의 비슷한 형태를 뛰고 있다. 하지만 세밀한 값의 분포도에서 차이가 있는 것을 알 수 있다.

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (20, 10))

ax[0].hist(hw[1173], bins = 17)
ax[0].set_title("1173")
ax[1].hist(hw[4502], bins = 17)
ax[1].set_title("4502")
ax[2].hist(hw[21370], bins = 17)
ax[2].set_title("21370")

In [ ]:
from collections import Counter

print(Counter(hw[1173]))
print()
print(Counter(hw[4502]))
print()
print(Counter(hw[21370]))

#### 1173과 4502를 키라고 가정하고 그 다음 연산에 있어서 상관 계수를 구해본다.

In [ ]:
real_result = np.int16(np.zeros(shape=(1,100000)))
fake_result = np.int16(np.zeros(shape=(1,100000)))

real_result[0] = fast_fqmul(1173, b[:,1])
real_result[0] = fast_fqmul(real_result[0], zeta[:,0])

fake_result[0] = fast_fqmul(4502, b[:,1])
fake_result[0] = fast_fqmul(fake_result[0], zeta[:,0])

In [ ]:
real_hw = np.zeros(shape=(1,100000), dtype = np.int16)
fake_hw = np.zeros(shape=(1,100000), dtype = np.int16)

In [ ]:
real_result[0]

In [ ]:
vf = np.vectorize(haming_weight)
real_hw = vf(real_result[0])
fake_hw = vf(fake_result[0])

In [ ]:
real_hw = real_hw.reshape(-1, 1)
fake_hw = fake_hw.reshape(-1, 1)

In [ ]:
total = cp.concatenate((cp.array(real_hw), cp.array(fake_hw), cp.array(traces)), axis=1)

cor = cp.corrcoef(total.T)

real_cor = cor[0,2:]
fake_cor = cor[1,2:]

#### 1173과 4502에 해당하는 키 값이 다음 연산에서 동일한 상관 계수를 갖는 것을 볼 수 있다.

In [ ]:
print("상관 계수가 첫 번째로 높은 값 : ", real_result[0][:25])
print()
print("상관 계수가 두 번째로 높은 값 : ", fake_result[0][:25])

In [ ]:
plt.figure(figsize=(20,15))
         
plt.plot(real_cor.get(), label = "1173", color = "blue", alpha = 0.5)      
plt.plot(fake_cor.get(), label = "4502", color = "red", alpha = 0.5)

plt.legend()
plt.show()

#### 다음으로 "r[0] += fqmul(a[0], b[0])" 연산에서 높은 상관 계수를 갖는 a[0]을 추측해 보자.

In [ ]:
real_a = np.int16(np.zeros(shape=(0xFFFF,100000)))
fake_a = np.int16(np.zeros(shape=(0xFFFF,100000)))

vf = np.vectorize(haming_weight)
b_0 = np.array(b[:, 0])

for key in tqdm(range(0xFFFF)):
    real_a[key] = fast_fqmul(key, b_0)
    fake_a[key] = fast_fqmul(key, b_0)

In [ ]:
for key in tqdm(range(0xFFFF)):
    real_a[key] += real_result[0]
    fake_a[key] += fake_result[0]

In [ ]:
real_a.shape

In [ ]:
real_hw = np.zeros(shape=(0xFFFF,100000), dtype = np.int16)
fake_hw = np.zeros(shape=(0xFFFF,100000), dtype = np.int16)

In [ ]:
vf = cp.vectorize(haming_weight)

for i in tqdm(range(50)):
    re = cp.array(real_a[:,i*2000:i*2000+2000])
    real_hw[:, i*2000:i*2000+2000] = vf(re).get()

In [ ]:
for i in tqdm(range(50)):
    re = cp.array(fake_a[:,i*2000:i*2000+2000])
    fake_hw[:, i*2000:i*2000+2000] = vf(re).get()

In [ ]:
hw_t.shape

In [ ]:
hw_t = real_hw.T

cor_result = []

trace_points = 600

limit = 100000
key_limit = 0xFFFF
for key in tqdm(range(0,key_limit,500)):
    if key <= 65000:
        temp_hw = hw_t[:,key*1:key*1+500]
        total = cp.concatenate((cp.array(temp_hw[:limit]), cp.array(traces[:limit])), axis = 1)
        cor = cp.corrcoef(total.T)
        print(cor.shape)
        temp = []
        for i in range(500):
            temp.append(cor[i][500:])
        temp = cp.array(temp)
        cor_result.append(temp)
    else:
        pass
        # temp_hw = hw_t[:,65000:].reshape(-1,1)
        # total = cp.concatenate((cp.array(temp_hw[:limit]), cp.array(traces[:limit])), axis = 1)
        # cor = cp.corrcoef(total.T)
        # print(cor.shape)
        # temp = cor[0][1:]
        # temp = cp.array(temp)
        # cor_result.append(temp)
cor_result = cp.array(cor_result)
cor_result.shape
cor_result = cor_result.reshape(-1, trace_points)

In [ ]:
cor_result = cp.array(cor_result)
cor_result.shape
cor_result = cor_result.reshape(-1, trace_points)

In [ ]:
plt.figure(figsize=(30,20))

for i in range(65000):
    if i==2435:
        pass
    else:
        plt.plot(cor_result[i,:].get(),alpha=0.3, color="gray")
plt.plot(cor_result[2435].get(), label = "2435", color = "red")        
plt.legend()
plt.show()

#### 실제키는 2435이다. 이때, 상관 계수가 -0.6 이상인 부분을 확인해 보자. 범위는 250 ~ 265라고 추정할 수 있다.

In [ ]:
cp.where(cor_result[2435] < -0.6)

#### 따라서, 250~265 사이에서 상관 계수가 -0.8 이상이 있는 값들을 뽑아 낸다. <br>해당하는 키가 총 8개 존재하는 것을 확인 할 수 있다. <br>이 중에서 실제 키가 가능한 것들을 조사해 보자.<br><br>이때도 실제 키 값을 갖고 있을 때 상관 계수가 가장 높은 것을 확인할 수 있다.<br>가능한 키는 2435와 5764인 것을 확인할 수 있다.

In [ ]:
print(cp.where(cp.nanmin(cor_result[:,250:266], axis = 1) < -0.8))

In [ ]:
cp.nanmin(cor_result[[ 2435,  5764,  9093, 12422, 54655, 57984, 61313, 64642], 250:266], axis=1)

In [ ]:
plt.figure(figsize=(30,20))

plt.plot(cor_result[64642].get(), label="64642", color = "blue", alpha = 0.5, linewidth = 4)
plt.plot(cor_result[2435].get(), label = "2435", color = "red", alpha = 0.5, linewidth = 4)        
plt.legend()
plt.show()

In [ ]:
hw_t = fake_hw.T

cor_result = []

trace_points = 600

limit = 100000
key_limit = 0xFFFF
for key in tqdm(range(0,key_limit,500)):
    if key <= 65000:
        temp_hw = hw_t[:,key*1:key*1+500]
        total = cp.concatenate((cp.array(temp_hw[:limit]), cp.array(traces[:limit])), axis = 1)
        cor = cp.corrcoef(total.T)
        print(cor.shape)
        temp = []
        for i in range(500):
            temp.append(cor[i][500:])
        temp = cp.array(temp)
        cor_result.append(temp)
    else:
        pass
        # temp_hw = hw_t[:,65000:].reshape(-1,1)
        # total = cp.concatenate((cp.array(temp_hw[:limit]), cp.array(traces[:limit])), axis = 1)
        # cor = cp.corrcoef(total.T)
        # print(cor.shape)
        # temp = cor[0][1:]
        # temp = cp.array(temp)
        # cor_result.append(temp)
cor_result = cp.array(cor_result)
cor_result.shape
cor_result = cor_result.reshape(-1, trace_points)

In [ ]:
cp.where(cor_result[2435] < -0.6)

In [ ]:
print(cp.where(cp.nanmin(cor_result[:,250:265], axis = 1) < -0.8))

In [ ]:
cp.nanmin(cor_result[[ 2435,  5764,  9093, 12422, 54655, 57984, 61313, 64642], 250:266], axis=1)

In [ ]:
plt.figure(figsize=(30,20))

plt.plot(cor_result[64642].get(), label="64642", color = "blue", alpha = 0.5, linewidth = 4)
plt.plot(cor_result[2435].get(), label = "2435", color = "red", alpha = 0.5, linewidth = 4)        
plt.legend()
plt.show()

위를 통해서도 상관 계수가 높은 것들이 많은 것을 볼 수 있다. 하지만 모든 경우에서 실제 키값을 갖는 경우에 높은 상관 계수를 갖는 것을 알 수 있다. 우리는 위의 과정을 통해서 a[1]의 키값으로는 1173과 4502이 가능한 것을 알 수 있고 a[0]의 키값으로는 2435와 64642이 되는 것을 알 수 있다.

### r[1] 값을 구해서 상관 계수를 구해보자. a[0]은 2435와 64642를 사용하자.

In [ ]:
real_result = np.int16(np.zeros(shape=(1,100000)))
fake_result = np.int16(np.zeros(shape=(1,100000)))

# r[1]을 구하는 식이다.
real_result[0] = fast_fqmul(2435, b[:,1])

fake_result[0] = fast_fqmul(64642, b[:,1])

In [ ]:
real_hw = np.zeros(shape=(1,100000), dtype = np.int16)
fake_hw = np.zeros(shape=(1,100000), dtype = np.int16)

vf = np.vectorize(haming_weight)
real_hw = vf(real_result[0])
fake_hw = vf(fake_result[0])

In [ ]:
real_hw = real_hw.reshape(-1, 1)
fake_hw = fake_hw.reshape(-1, 1)

In [ ]:
total = cp.concatenate((cp.array(real_hw), cp.array(fake_hw), cp.array(traces)), axis=1)

cor = cp.corrcoef(total.T)

real_cor = cor[0,2:]
fake_cor = cor[1,2:]

* r[0] += fqmul(a[0], b[0]);
* r[1]  = fqmul(a[0], b[1]);

위의 두 과정에서 a[0]이 사용되었고 우리는 2435와 64642 둘 중에서 키 값이 될 수 있다는 것을 확인했다. 그런데 두 상황에서 모두 2435의 상관 계수가 높기 때문에 이 값이 실제 키 값일 것이다.

In [ ]:
plt.figure(figsize=(20,15))

# 두 계산 모두에서 2435의 키값이 더욱 높은 상관 계수를 갖는 것을 확인했다.
plt.plot(real_cor.get(), label = "2435", color = "red", alpha = 0.5, linewidth = 4)      
plt.plot(fake_cor.get(), label = "64642", color = "blue", alpha = 0.5, linewidth = 4)

plt.legend()
plt.show()

### 2435를 키라고 가정하고 위에서 정하지 못한 a[1] 값을 정해 보자. a[1]은 1173, 4502 둘 중에서 하나이다.

In [ ]:
# 1173으로 가정한 것이 real_result가 된다.
# fake_result는 4502로 가정한다.
fake_result = real_result

b_0 = np.array(b[:, 0])

real_result += fast_fqmul(1173, b_0)
fake_result += fast_fqmul(4502, b_0)

In [ ]:
real_hw = np.zeros(shape=(1,100000), dtype = np.int16)
fake_hw = np.zeros(shape=(1,100000), dtype = np.int16)

vf = np.vectorize(haming_weight)
real_hw = vf(real_result[0])
fake_hw = vf(fake_result[0])

In [ ]:
real_hw = real_hw.reshape(-1, 1)
fake_hw = fake_hw.reshape(-1, 1)

total = cp.concatenate((cp.array(real_hw), cp.array(fake_hw), cp.array(traces)), axis=1)

cor = cp.corrcoef(total.T)

real_cor = cor[0,2:]
fake_cor = cor[1,2:]

In [ ]:
sum(real_hw == fake_hw)

In [ ]:
plt.figure(figsize=(20,15))

plt.plot(real_cor.get(), label = "1173", color = "red", alpha = 0.5, linewidth = 4)      
plt.plot(fake_cor.get(), label = "4502", color = "blue", alpha = 0.5, linewidth = 4)

plt.legend()
plt.show()

In [ ]:
print(cp.nanmin(real_cor))
print(cp.nanmin(fake_cor))

In [ ]:
b_0 = np.array(b[:, 0])

for key in tqdm(range(0xFFFF)):
    real_a[key] = fast_fqmul(key, b_0)
    fake_a[key] = fast_fqmul(key, b_0)


r[0]  = fqmul(np.int16(a[1]), np.int16(b[1]));
    r[0]  = fqmul(r[0], zeta);
    r[0] += fqmul(a[0], b[0]);`
    r[1]  = fqmul(a[0], b[1]);
    r[1] += fqmul(a[1], b[0]);